<h1 align="center">SYMPSENSE</h1>
<h3 align="center">A Micro Health Multi-Agent Symptom Insight System</h3>

## Abstract  
Welcome to my capstone project for the Kaggle × Google 5-Day Agents Intensive.  

**SympSense** is a small, safe, multi-agent system that analyzes short symptom descriptions, runs specialist sub-agents (symptom extraction, triage, suggestions, trend detection), and produces a structured, non-diagnostic triage summary.

This notebook presents a complete, end-to-end demonstration of how multiple specialized AI agents can work together in a safe, modular, and well-structured pipeline to analyze and solve a real-world problem of health-related symptom descriptions.

The goal is not just medical diagnosis, but to showcase how to solve a problem using agents:
- Multi-agent architecture  
- Memory usage  
- Synchronous pipeline orchestration
- Gemini/ADK integration  
- Clean reproducible logic with deterministic fallbacks  

As we walk through this notebook, each cell will build a small, independent piece of the system - eventually assembling a complete multi-agent triage workflow.


## Objectives
- Build a **multi-agent pipeline** that demonstrates agent orchestration, memory, and safe decision-making.
- Use **Google ADK / Gemini** for model-assisted parsing and re-phrasing, while keeping **deterministic fallbacks** as the authoritative source for safety-critical outputs.
- Produce a **clean console demo** suitable to show inputs, intermediate agent outputs, and final structured result.
- Keep the implementation self-contained, dependency-light, and reproducible (deterministic mode for testing).


## Key features
- **Specialist sub-agents:** SymptomAgent, TriageAgent, SuggestionAgent, TrendAgent.
- **Shared memory:** stores user/agent/model messages and supports simple trend analysis.
- **Model wrapper:** robust ADK/Gemini invocation wrapper that extracts JSON and validates outputs.
- **Synchronous orchestration:** symptom extraction first, then parallelized triage/suggestion/trend agents; deterministic triage is authoritative for safety.
- **Print-only demo:** console logs and an optional `memory.json` file to show persistent state across runs.
- **Deterministic fallback:** every agent has rule-based behavior so the notebook runs without Gemini (recommended for reproducible tests and grading).
- **Safety disclaimer:** explicit non-diagnostic language and escalation tips for urgent symptoms.


## Notebook structure
1. Kaggle secrets setup (Cell 1)
2. Imports, config & ADK detection (Cell 2)
3. Memory (Cell 3)
4. Model wrapper (Cell 4)
5. Symptom knowledge base (Cell 5)
6. Sub-agents (Cell 6)
7. MultiAgentCoordinator (Cell 7)
8. Demo runner (Cell 8)
9. Quick tests (Cell 9)
10. Project Summary & Next steps (Cell 10)


## Safety note
> **This project is NOT a medical diagnostic tool.** It provides general, non-diagnostic suggestions and triage. For any emergency (severe shortness of breath, uncontrolled bleeding, chest pain, loss of consciousness, etc.) the user should seek immediate professional medical help.


# 1. Setting up environment & secrets

Before we build agents, we prepare the environment.  
This notebook supports two modes:

**1. Deterministic mode (recommended for testing & evaluation)**  
Runs entirely on rule-based logic with *no external dependencies*.

**2. Gemini-assisted mode**  
If Google ADK and a valid `GOOGLE_API_KEY` are available, certain agents may request structured outputs from Gemini to enrich their reasoning.

In the first cell, we attempt to load your API key from Kaggle Secrets (only needed if Gemini mode is enabled later).  
If no key is found, the system will still run perfectly in deterministic mode.

**Instructions**
1. If you plan to use Gemini in this notebook, add a Kaggle secret named `GOOGLE_API_KEY`:
   - Open Kaggle → Notebook settings → Add-ons → Secrets → Add secret.
2. If running locally, set environment variable `GOOGLE_API_KEY` before starting your notebook:
   - macOS/Linux: `export GOOGLE_API_KEY="..."`  
   - Windows (PowerShell): `$env:GOOGLE_API_KEY="..."`
3. If you do not have ADK or do not want to use Gemini, you can leave the key unset and run deterministically.


In [167]:
# Kaggle secrets setup

# Instructions:
# 1) If you are running on Kaggle, add a secret named 'GOOGLE_API_KEY' via Kaggle -> Notebook settings -> Add-ons -> Secrets -> Add secret.
# 2) If running locally, set env var GOOGLE_API_KEY before starting the notebook:
#    export GOOGLE_API_KEY="your_key"
# 3) Toggle USE_GEMINI in Cell 2 to True to enable ADK calls (only if ADK is installed and key present).
# 4) Run cells top-to-bottom.

import os
print("Notebook header cell loaded. Make sure you set 'GOOGLE_API_KEY' in Kaggle secrets or environment if you plan to use Gemini.")
try:
    # Kaggle secrets helper — only available in Kaggle environment
    from kaggle_secrets import UserSecretsClient
    try:
        key = UserSecretsClient().get_secret("GOOGLE_API_KEY")
        if key:
            os.environ["GOOGLE_API_KEY"] = key
            print("✅ Loaded GOOGLE_API_KEY from Kaggle secrets.")
    except Exception as e:
        print("⚠️ Could not load from Kaggle secrets or secret not set:", e)
except Exception:
    print("ℹ️ kaggle_secrets not available (not running on Kaggle). Check environment variable GOOGLE_API_KEY.")

Notebook header cell loaded. Make sure you set 'GOOGLE_API_KEY' in Kaggle secrets or environment if you plan to use Gemini.
✅ Loaded GOOGLE_API_KEY from Kaggle secrets.


# 2. Configuration & Gemini/ADK activation

Here we define global switches:
- `USE_GEMINI = False` keeps the project fully deterministic.
- `USE_GEMINI = True` will attempt to load and initialize the Google ADK Agent interface.

This cell also imports all core Python modules and checks whether the notebook runtime supports ADK.  
If ADK isn't installed, the system gracefully falls back to deterministic logic ensuring the notebook works everywhere, including Kaggle Free Tier runtimes.

This design guarantees full reproducibility regardless of environment.

**Instructions**
- Set `USE_GEMINI = True` only if:
  1. You installed the ADK libraries in this kernel, and
  2. `GOOGLE_API_KEY` is set (via Kaggle secrets or environment).
- Otherwise keep `USE_GEMINI = False` for a deterministic offline demo.



In [168]:
# Imports, config, ADK detection
# Toggle USE_GEMINI to True only if you installed ADK and have GOOGLE_API_KEY set in environment.

USE_GEMINI = True   # <-- set True to enable ADK/Gemini usage (only if ADK is installed and key present)
GEMINI_MODEL_NAME = "gemini-2.5-flash-lite"
MAX_WORKERS = 4      # concurrency for parallel sub-agents

import os, json, re, time
from dataclasses import dataclass, field
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional, Callable, Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed

# Try to import ADK/Gemini only if user requested it (protects Kaggle default kernels)
ADK_AVAILABLE = False
root_agent = None

if USE_GEMINI:
    try:
        from google.adk.agents import Agent
        from google.adk.models.google_llm import Gemini
        
        retry_config = None
        root_agent = Agent(
            name="capstone_root_agent",
            model=Gemini(model=GEMINI_MODEL_NAME, retry_options=retry_config),
            description="Root ADK agent for multi-agent demo",
            instruction="When asked to return structured JSON, return JSON only."
        )
        ADK_AVAILABLE = True
        print("✅ ADK/Gemini loaded successfully. ADK_AVAILABLE=True")
    except Exception as e:
        ADK_AVAILABLE = False
        root_agent = None
        print("⚠️ ADK/Gemini failed to import or initialize. Set USE_GEMINI=False to run deterministically. Error:", e)
else:
    print("ℹ️ USE_GEMINI is False — running deterministically (no model calls).")


✅ ADK/Gemini loaded successfully. ADK_AVAILABLE=True


# 3. Conversation Memory (shared across all agents)

To simulate a more realistic agent ecosystem, we maintain a lightweight memory system.  
This allows agents to detect symptom recurrence over time and provide trend-based insights.

Our memory:
- Stores each user message  
- Stores every agent output  
- Can persist across runs (optional file saving)  
- Can limit history to keep things efficient  

This layer is foundational for the TrendAgent later in the pipeline.

In [169]:
# Memory class (shared across agents)

@dataclass
class Memory:
    messages: List[Dict[str, Any]] = field(default_factory=list)
    persist_file: Optional[str] = "multi_agent_memory.json"
    max_history: int = 500

    def add(self, role: str, content: str):
        entry = {"role": role, "content": content, "time": datetime.now().isoformat()}
        self.messages.append(entry)
        if len(self.messages) > self.max_history:
            self.messages = self.messages[-self.max_history:]
        print(f"[Memory] {role}: {content}")

    def recent_user_reports(self, hours: int = 168):
        cutoff = datetime.now() - timedelta(hours=hours)
        return [m for m in self.messages if m["role"] == "user" and datetime.fromisoformat(m["time"]) >= cutoff]

    def persist(self):
        if not self.persist_file:
            return
        try:
            with open(self.persist_file, "w", encoding="utf-8") as f:
                json.dump(self.messages, f, indent=2, ensure_ascii=False)
            print(f"[Memory] persisted {len(self.messages)} entries -> {self.persist_file}")
        except Exception as e:
            print("[Memory] persist error:", e)

    def load(self):
        if not self.persist_file:
            return
        try:
            with open(self.persist_file, "r", encoding="utf-8") as f:
                self.messages = json.load(f)
            print(f"[Memory] loaded {len(self.messages)} entries from {self.persist_file}")
        except FileNotFoundError:
            print("[Memory] no existing memory file found.")
        except Exception as e:
            print("[Memory] load error:", e)


# 4. Model Wrapper - safe interface for Gemini output

For environments where Gemini is available, we use a robust wrapper that:
- Sends a structured prompt
- Extracts valid JSON from the model response
- Validates the returned structure
- Falls back safely if anything is malformed

Even when Gemini is active, **deterministic logic always remains authoritative** for any safety-critical decision such as triage severity.

This dual-mode design keeps the system reliable, transparent, and predictable.


In [171]:
# ModelWrapper: robust JSON extraction and ADK invocation wrapper

class ModelWrapper:
    @staticmethod
    def extract_json_candidate(text: str) -> Optional[dict]:
        """Attempt to robustly extract a JSON object from a string."""
        if not text:
            return None
        if "{" in text and "}" in text:
            first = text.find("{")
            last = text.rfind("}")
            candidate = text[first:last+1]
            try:
                return json.loads(candidate)
            except Exception:
                # brute-force search for any JSON object substrings
                for i in range(len(text)):
                    if text[i] == "{":
                        for j in range(len(text)-1, i, -1):
                            if text[j] == "}":
                                cand = text[i:j+1]
                                try:
                                    return json.loads(cand)
                                except Exception:
                                    continue
        return None

    @staticmethod
    def call_agent(agent, prompt: str, validate_fn: Callable[[dict], bool], timeout_s: int = 12) -> Optional[dict]:
        """Call ADK Agent safely and parse/validate JSON. Returns dict or None."""
        if agent is None or not ADK_AVAILABLE:
            print("[ModelWrapper] ADK not available or agent is None.")
            return None
        try:
            raw = None
            try:
                if hasattr(agent, "run"):
                    raw = agent.run(prompt)
                elif hasattr(agent, "invoke"):
                    raw = agent.invoke(prompt)
                else:
                    raw = agent(prompt)
            except Exception as inner:
                
                return None
            resp_text = str(raw)
            parsed = ModelWrapper.extract_json_candidate(resp_text)
            if parsed is None:
                # model returned non-JSON — allow validator to inspect 'raw' if it wants
                candidate = {"raw": resp_text}
                if validate_fn(candidate):
                    return candidate
                return None
            if validate_fn(parsed):
                print("[ModelWrapper] model returned valid JSON.")
                return parsed
            else:
                print("[ModelWrapper] model JSON failed validation.")
                return None
        except Exception as e:
            print("[ModelWrapper] unexpected error:", e)
            return None


# 5. Symptom Knowledge Base

Before calling any model or agent, we must define our ground truth.  
This cell introduces a small but extensible **rule-based symptom matcher**, mapping keywords to canonical symptom IDs.

It also stores basic, non-medical suggestions and precautions for each symptom used later by the SuggestionAgent.

This is the deterministic backbone of the entire system.

In [172]:
# Symptom knowledge base (SymptomKB)

class SymptomKB:
    def __init__(self):
        self.keyword_map = {
            "fever": [r"\bfever\b", r"\btemperature\b", r"\bchills\b", r"\bhot\b"],
            "cough": [r"\bcough\b", r"\bcoughing\b"],
            "headache": [r"\bheadache\b", r"\bmigraine\b"],
            "sore_throat": [r"\bsore throat\b", r"\bthroat pain\b"],
            "nausea": [r"\bnausea\b", r"\bvomit\b"],
            "shortness_of_breath": [r"\bshortness of breath\b", r"\bbreathless\b", r"\bcan't breathe\b"],
            "rash": [r"\brash\b", r"\bred spots\b"],
            "dizziness": [r"\bdizzy\b", r"\bdizziness\b"],
            "bleeding": [r"\bbleed\b", r"\bbleeding\b"],
            "fatigue": [r"\btired\b", r"\bfatigued\b", r"\bfatigue\b"]
        }
        self.advice = {
            "fever": {"precautions": ["Rest","Hydrate"], "suggestions": ["Paracetamol if suitable"], "urgent_if":[">39C",">72h"]},
            "shortness_of_breath": {"precautions":["Stop exertion"], "suggestions":["Seek immediate care"], "urgent_if":["severe breathlessness"]},
            # add more entries as needed
        }

    def match(self, text: str) -> List[str]:
        found = []
        low = text.lower()
        for k, pats in self.keyword_map.items():
            for p in pats:
                if re.search(p, low):
                    found.append(k)
                    break
        return found

    def advice_for(self, symptom: str) -> Dict[str, List[str]]:
        return self.advice.get(symptom, {"precautions":["Rest"], "suggestions":["Monitor"], "urgent_if": []})


# 6. Defining our specialist agents

In a multi-agent architecture, each agent should have a single responsibility.  
This cell defines four lightweight sub-agents:

### **1. SymptomAgent**  
Extracts canonical symptoms from the user’s text.  
If Gemini is available, it may refine extraction otherwise rule-based matching is used.

### **2. TriageAgent**  
Categorizes severity into: none → mild → moderate → urgent.  
Deterministic triage is the final authoritative decision.

### **3. SuggestionAgent**  
Provides general wellness tips and precautions (not medical advice).  

### **4. TrendAgent**  
Looks into past memory to detect repeated symptoms or patterns.

Each agent follows the same template:
- Optional Gemini prompt  
- Strict JSON validation  
- Deterministic fallback logic  

This modular layout makes the system easy to extend and safe to deploy.


In [173]:
# SubAgent base class and concrete agents (SymptomAgent, TriageAgent, SuggestionAgent, TrendAgent)

class SubAgent:
    def __init__(self, name: str, memory: Memory, use_gemini: bool = False, model_agent=None):
        self.name = name
        self.memory = memory
        self.use_gemini = use_gemini and ADK_AVAILABLE
        self.model_agent = model_agent

    def prompt_and_validate(self, user_text: str) -> Tuple[str, Callable[[dict], bool]]:
        raise NotImplementedError

    def deterministic(self, user_text: str, context: Dict[str, Any]) -> Dict[str, Any]:
        raise NotImplementedError

    def run(self, user_text: str, context: Dict[str, Any]) -> Dict[str, Any]:
        print(f"[{self.name}] start")
        prompt, validate_fn = self.prompt_and_validate(user_text)
        model_output = None
        if self.use_gemini and self.model_agent is not None:
            model_output = ModelWrapper.call_agent(self.model_agent, prompt, validate_fn)
            if model_output is not None:
                print(f"[{self.name}] model returned: {model_output}")
        out = {"from_model": bool(model_output), "model": model_output}
        det = self.deterministic(user_text, context)
        out["deterministic"] = det
        print(f"[{self.name}] done (deterministic -> {det})")
        return out

# SymptomAgent
class SymptomAgent(SubAgent):
    def __init__(self, memory: Memory, kb: SymptomKB, use_gemini: bool=False, model_agent=None):
        super().__init__("SymptomAgent", memory, use_gemini, model_agent)
        self.kb = kb

    def prompt_and_validate(self, user_text: str):
        prompt = (
            "Return JSON only: { 'symptoms': [list of canonical symptom ids like 'fever', 'cough'] }\n"
            f"User message: '''{user_text}'''"
        )
        def validate(d): return isinstance(d, dict) and ("symptoms" in d and isinstance(d["symptoms"], list))
        return prompt, validate

    def deterministic(self, user_text: str, context: Dict[str, Any]):
        syms = self.kb.match(user_text)
        return {"symptoms": syms}

# TriageAgent
class TriageAgent(SubAgent):
    def __init__(self, memory: Memory, kb: SymptomKB, use_gemini: bool=False, model_agent=None):
        super().__init__("TriageAgent", memory, use_gemini, model_agent)
        self.kb = kb

    def prompt_and_validate(self, user_text: str):
        prompt = (
            "Return JSON only: { 'severity': 'none'|'mild'|'moderate'|'urgent', 'reason': str }\n"
            f"User message: '''{user_text}'''"
        )
        def validate(d): return isinstance(d, dict) and ("severity" in d and d["severity"] in {"none","mild","moderate","urgent"})
        return prompt, validate

    def deterministic(self, user_text: str, context: Dict[str, Any]):
        syms = context.get("symptoms", [])
        sset = set(syms)
        if "shortness_of_breath" in sset or "bleeding" in sset:
            return {"severity": "urgent", "reason": "critical symptom present"}
        if len(sset) >= 3:
            return {"severity": "moderate", "reason": "multiple symptoms"}
        if sset:
            return {"severity": "mild", "reason": "single or mild symptom"}
        return {"severity": "none", "reason": "no symptoms detected"}

# SuggestionAgent
class SuggestionAgent(SubAgent):
    def __init__(self, memory: Memory, kb: SymptomKB, use_gemini: bool=False, model_agent=None):
        super().__init__("SuggestionAgent", memory, use_gemini, model_agent)
        self.kb = kb

    def prompt_and_validate(self, user_text: str):
        prompt = (
            "Return JSON only: { 'suggestions': [strings], 'precautions': [strings] }\n"
            f"User message: '''{user_text}'''"
        )
        def validate(d): return isinstance(d, dict) and ("suggestions" in d and isinstance(d["suggestions"], list))
        return prompt, validate

    def deterministic(self, user_text: str, context: Dict[str, Any]):
        syms = context.get("symptoms", [])
        suggestions = []
        precautions = []
        for s in syms:
            advice = self.kb.advice_for(s)
            suggestions.extend(advice.get("suggestions", []))
            precautions.extend(advice.get("precautions", []))
        suggestions = list(dict.fromkeys(suggestions))
        precautions = list(dict.fromkeys(precautions))
        return {"suggestions": suggestions, "precautions": precautions}

# TrendAgent
class TrendAgent(SubAgent):
    def __init__(self, memory: Memory, kb: SymptomKB, use_gemini: bool=False, model_agent=None):
        super().__init__("TrendAgent", memory, use_gemini, model_agent)
        self.kb = kb

    def prompt_and_validate(self, user_text: str):
        prompt = (
            "Return JSON only: { 'trend_explanation': str }\n"
            f"User message: '''{user_text}'''"
        )
        def validate(d): return isinstance(d, dict) and ("trend_explanation" in d)
        return prompt, validate

    def deterministic(self, user_text: str, context: Dict[str, Any]):
        recent = self.memory.recent_user_reports(hours=168)
        counts = {}
        for r in recent:
            found = self.kb.match(r["content"])
            for f in found:
                counts[f] = counts.get(f, 0) + 1
        msgs = [f"{s} mentioned {c} times" for s,c in counts.items()] or ["No persistent trends found in the past week."]
        return {"trend_explanation": "; ".join(msgs), "counts": counts}


# 7. Multi-Agent Coordinator (synchronous orchestration)

This cell unifies everything into a complete flow.

The Coordinator controls the workflow:
1. **Step 1 - Symptom extraction:**  
   Runs first so other agents can use its output as shared context.

2. **Step 2 - Parallel execution:**  
   TriageAgent, SuggestionAgent, and TrendAgent run side-by-side using Python’s ThreadPoolExecutor.

3. **Step 3 - Merge & finalize results:**  
   - Deterministic triage determines the severity  
   - Suggestions & precautions are combined  
   - Trend insights are included  
   - A safety disclaimer is appended  

The coordinator behaves like an “agent conductor,” ensuring every specialist contributes to the final structured triage summary.

This is where the real multi-agent intelligence emerges.


In [174]:
# MultiAgentCoordinator orchestration (runs sub-agents synchronously but parallelizes where useful)

class MultiAgentCoordinator:
    def __init__(self, use_gemini: bool=False, memory_file: Optional[str]=None):
        self.memory = Memory(persist_file=memory_file)
        self.kb = SymptomKB()
        self.symptom_agent = SymptomAgent(self.memory, self.kb, use_gemini=use_gemini, model_agent=root_agent)
        self.triage_agent = TriageAgent(self.memory, self.kb, use_gemini=use_gemini, model_agent=root_agent)
        self.suggestion_agent = SuggestionAgent(self.memory, self.kb, use_gemini=use_gemini, model_agent=root_agent)
        self.trend_agent = TrendAgent(self.memory, self.kb, use_gemini=use_gemini, model_agent=root_agent)
        self.agents = [self.symptom_agent, self.triage_agent, self.suggestion_agent, self.trend_agent]

    def run_pipeline(self, user_text: str, verbose: bool=True) -> Dict[str, Any]:
        if verbose:
            print("\n[Coordinator] Received:", user_text)
        self.memory.add("user", user_text)

        # Step 1: symptom extraction (run synchronously first to create context)
        sym_out = self.symptom_agent.run(user_text, {})
        if sym_out.get("from_model") and isinstance(sym_out["model"], dict) and "symptoms" in sym_out["model"]:
            symptoms = sym_out["model"]["symptoms"]
        else:
            symptoms = sym_out["deterministic"]["symptoms"]

        context = {"symptoms": symptoms}
        if verbose:
            print("[Coordinator] Extracted symptoms:", symptoms)

        # Step 2: parallel triage, suggestions, trends
        agents_to_run = [self.triage_agent, self.suggestion_agent, self.trend_agent]
        results = {}
        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
            futures = {ex.submit(ag.run, user_text, context): ag for ag in agents_to_run}
            for fut in as_completed(futures):
                ag = futures[fut]
                try:
                    res = fut.result()
                    results[ag.name] = res
                    if verbose:
                        print(f"[Coordinator] {ag.name} finished.")
                except Exception as e:
                    results[ag.name] = {"error": str(e)}
                    print(f"[Coordinator] {ag.name} failed:", e)

        triage_det = results.get("TriageAgent", {}).get("deterministic", {})
        final_severity = triage_det.get("severity", "none")
        final_reason = triage_det.get("reason", "")

        suggestions = results.get("SuggestionAgent", {}).get("deterministic", {}).get("suggestions", [])
        precautions = results.get("SuggestionAgent", {}).get("deterministic", {}).get("precautions", [])
        trends = results.get("TrendAgent", {}).get("deterministic", {}).get("counts", {})

        final = {
            "symptoms": symptoms,
            "severity": final_severity,
            "severity_reason": final_reason,
            "precautions": precautions,
            "suggestions": suggestions,
            "trends": trends,
            "disclaimer": "This is not medical advice. For emergencies, seek immediate medical attention."
        }

        # memory & persist
        self.memory.add("agent", json.dumps(final))
        self.memory.persist()

        if verbose:
            print("[Coordinator] Final result:", json.dumps(final, indent=2, ensure_ascii=False))
        return final


# 8. Running the demo - sample symptom inputs

In this cell, we test the full pipeline using a few representative symptom descriptions.

You’ll see:
- Memory updating  
- Each agent logging its reasoning  
- Parallel agent execution  
- A clean final JSON-style result  

This demonstration is ideal because it clearly reveals how each agent collaborates to produce the final output.

In [175]:
# Demo runner: sample inputs to exercise the multi-agent pipeline

if __name__ == "__main__" or True:
    print("=== Multi-agent synchronous demo ===")
    # Toggle use_gemini to True if you set USE_GEMINI=True in Cell 2 AND ADK is available
    coord = MultiAgentCoordinator(use_gemini=(USE_GEMINI and ADK_AVAILABLE), memory_file=None)
    samples = [
        "I've had a fever and headache for 2 days and feel very tired.",
        "Now I'm breathless and can't breathe and having chest tightness.",
        "I noticed a rash and a little dizziness after taking new medicine."
    ]
    for s in samples:
        out = coord.run_pipeline(s, verbose=True)
        print("-" * 60)
        time.sleep(0.4)


=== Multi-agent synchronous demo ===

[Coordinator] Received: I've had a fever and headache for 2 days and feel very tired.
[Memory] user: I've had a fever and headache for 2 days and feel very tired.
[SymptomAgent] start
[SymptomAgent] done (deterministic -> {'symptoms': ['fever', 'headache', 'fatigue']})
[Coordinator] Extracted symptoms: ['fever', 'headache', 'fatigue']
[TriageAgent] start
[TriageAgent] done (deterministic -> {'severity': 'moderate', 'reason': 'multiple symptoms'})
[SuggestionAgent] start
[SuggestionAgent] done (deterministic -> {'suggestions': ['Paracetamol if suitable', 'Monitor'], 'precautions': ['Rest', 'Hydrate']})
[TrendAgent] start
[TrendAgent] done (deterministic -> {'trend_explanation': 'fever mentioned 1 times; headache mentioned 1 times; fatigue mentioned 1 times', 'counts': {'fever': 1, 'headache': 1, 'fatigue': 1}})
[Coordinator] TriageAgent finished.
[Coordinator] SuggestionAgent finished.
[Coordinator] TrendAgent finished.
[Memory] agent: {"symptoms": 

# 9. Quick deterministic validation tests

This cell performs light automatic checks to verify that:
- Fever is detected  
- Breathlessness triggers urgent triage  
- The system behaves predictably in deterministic mode  

These tests can be expanded later, but even these simple checks reinforce that the pipeline is reliable and safe.


In [176]:
# Quick unit-test-like checks (deterministic; set USE_GEMINI=False for reproducible tests)

def quick_tests():
    print("\n=== Running quick deterministic tests ===")
    coord = MultiAgentCoordinator(use_gemini=False, memory_file=None)
    r1 = coord.run_pipeline("I have a fever for 3 days", verbose=False)
    assert "fever" in r1["symptoms"] or r1["severity"] != "none"
    r2 = coord.run_pipeline("I'm breathless and can't breathe", verbose=False)
    assert r2["severity"] == "urgent" or "shortness_of_breath" in r2["symptoms"]
    print("Quick deterministic tests passed.")

quick_tests()



=== Running quick deterministic tests ===
[Memory] user: I have a fever for 3 days
[SymptomAgent] start
[SymptomAgent] done (deterministic -> {'symptoms': ['fever']})
[TriageAgent] start
[TriageAgent] done (deterministic -> {'severity': 'mild', 'reason': 'single or mild symptom'})
[SuggestionAgent] start
[SuggestionAgent] done (deterministic -> {'suggestions': ['Paracetamol if suitable'], 'precautions': ['Rest', 'Hydrate']})
[TrendAgent] start
[TrendAgent] done (deterministic -> {'trend_explanation': 'fever mentioned 1 times', 'counts': {'fever': 1}})
[Memory] agent: {"symptoms": ["fever"], "severity": "mild", "severity_reason": "single or mild symptom", "precautions": ["Rest", "Hydrate"], "suggestions": ["Paracetamol if suitable"], "trends": {"fever": 1}, "disclaimer": "This is not medical advice. For emergencies, seek immediate medical attention."}
[Memory] user: I'm breathless and can't breathe
[SymptomAgent] start
[SymptomAgent] done (deterministic -> {'symptoms': ['shortness_of_b

# 10. Project summary & next steps

Congratulations! At this point, we've assembled a complete multi-agent system.

This project demonstrates:
- Clean agent modularity  
- Deterministic safety with model assistance  
- Context sharing through memory  
- Synchronous multi-agent orchestration  
- Clear structured output ready for evaluation or extension  

### Next Steps
- Expand the Symptom Knowledge Base
- Add new agents (e.g., RiskFactorAgent or HabitAgent)
- Deploy as a small backend API (FastAPI/Firebase)

Thank you for reviewing this capstone project!